This notebook tests the cooling rate and heating rate by using the switch valve. 
The hot water bath = 75 degc
The cold water bath = 35 degc
300 rpm
50mL water

In [1]:
%cd ..

C:\Users\wuyua\Projects\PycharmProjects\RxExperiments


In [2]:
import numpy as np
import pandas as pd
import rx
from rx import operators
from rx import scheduler
from rx import subject
from rx import disposable

import logging
import datetime
from plotly import graph_objects as go
from plotly.subplots import make_subplots
from utils.mqtt_wrapper import MQTTClientWrapper

from controls.mqtt_switch_valve import MQTTSwitchValve
from controls.fp50 import FP50Control, FP50Message, FP50Command
from sources.mqtt_ds18 import MQTTDS18Source
from sinks.save_data_sink import SaveDataSink
from sinks.visualization_sink import PlotlyVisualizationSink
from operators import data_framer

from configs.config import GlobalConfig
GlobalConfig.initialize_global_configuration("configs/defaults.json")

from utils.auto_disposable import AutoDisposable
from utils.logging import configure_logger_to_output
logging_output = configure_logger_to_output(level=logging.INFO)

In [3]:
# MQTT connection
client = MQTTClientWrapper("experiment_switch_valve")
client.connect("192.168.43.1")
client.loop_start()

In [4]:
ds18 = MQTTDS18Source(client)
ds18_save = SaveDataSink("ds18", "value", auto_timestamp = True)
ds18_save_subs = AutoDisposable(ds18.pipe(
    operators.map(lambda x: pd.DataFrame(data=[{"value": x["value"]}])),
).subscribe(ds18_save))
ds18_plot = PlotlyVisualizationSink(name="Temperature")
ds18_plot_subs = AutoDisposable(ds18.pipe(
    operators.map(lambda x: pd.DataFrame(data=[{"value": x["value"]}])),
    data_framer.data_framer(window_length = 300, auto_timestamp = True),
    operators.map(lambda x: [go.Scatter(x=x.timestamp, y=x.value, name="Temperature")]),
).subscribe(ds18_plot))

In [5]:
ds18_plot.figure

FigureWidget({
    'data': [], 'layout': {'template': '...', 'title': {'text': 'Temperature'}}
})

In [6]:
valve = MQTTSwitchValve(client)
valve_save = SaveDataSink("valve", "state", auto_timestamp = True)
valve_save_subs = AutoDisposable(valve.pipe(
    operators.map(lambda x: pd.DataFrame(data=[{"state": x}])),
).subscribe(valve_save))
valve_plot = PlotlyVisualizationSink(name="Valve state")
valve_plot_subs = AutoDisposable(valve.pipe(
    operators.map(lambda x: pd.DataFrame(data=[{"state": x}])),
    data_framer.data_framer(window_length = 300, auto_timestamp = True),
    operators.map(lambda x: [go.Scatter(x=x.timestamp, y=x.state, name="Valve State")]),
).subscribe(valve_plot))

In [7]:
valve_plot.figure

FigureWidget({
    'data': [{'name': 'Valve State',
              'type': 'scatter',
              'uid': '34f…

In [8]:
fp50 = FP50Control(client)
fp50_df = fp50.pipe(
    operators.map(lambda x: pd.DataFrame(data=[{"power": x.power, "setpoint": x.setpoint, "temperature": x.temperature}]))
)
fp50_save = SaveDataSink("fp50", "data", auto_timestamp = True)

fp50_save_subs = AutoDisposable(fp50_df.subscribe(fp50_save))
fp50_power_plot = PlotlyVisualizationSink(name="Power")
fp50_power_plot_subs = AutoDisposable(fp50_df.pipe(
    data_framer.data_framer(window_length = 300, auto_timestamp = True),
    operators.map(lambda x: [go.Scatter(x=x.timestamp, y=x.power, name="Power")]),
).subscribe(fp50_power_plot))

fp50_temperature_plot = PlotlyVisualizationSink(name="Temperature")
fp50_temperature_plot_subs = AutoDisposable(fp50_df.pipe(
    data_framer.data_framer(window_length = 300, auto_timestamp = True),
    operators.map(lambda x: [go.Scatter(x=x.timestamp, y=x.temperature, name="Temperature")]),
).subscribe(fp50_temperature_plot))

fp50.on_command(FP50Command(35)) # This is required to make the plots update!

In [9]:
fp50_power_plot.figure

FigureWidget({
    'data': [], 'layout': {'template': '...', 'title': {'text': 'Power'}}
})

In [10]:
fp50_temperature_plot.figure

FigureWidget({
    'data': [], 'layout': {'template': '...', 'title': {'text': 'Temperature'}}
})

In [11]:
logging_output

Output(outputs=({'output_type': 'stream', 'name': 'stdout', 'text': '2020-10-16 04:33:55 \x1bINFO  \x1b…

In [12]:
src = rx.interval(30)
valve_ctrl_1_subs = AutoDisposable(src.pipe(operators.map(lambda x: "1")).subscribe(valve))
valve_ctrl_2_subs = AutoDisposable(src.pipe(operators.delay(15), operators.map(lambda x: "0")).subscribe(valve))

In [13]:
src.subscribe(print)

In [21]:
a = rx.interval(1.0).subscribe(logging.info)

In [22]:
a.dispose()